In [7]:
import time

from env import Env
from service.file_service import FileService

############### Applcation Main ###############
# 애플리케이션 진입점
def main():
    print('디렉터리 감시를 통한 보안 위협 탐지 프로그램 - 장상훈')
    env = Env()                                         # 환경변수 로드
    fileService = FileService(env)                      # 파일 입출력 서비스 클래스
    fileInfoList = fileService.getFileInfoList()        # 디렉터리 내 파일 정보 초기화
    snapHashDict = {}                                   # 횬쟈 폴더 내 해시값 스냅샷
    while True:
        fileInfoList = fileService.getFileInfoList()    # 파일 목록 최신화

        # 현재 폴더 내 유효한 해시값
        currentHashDict = {f"{fileInfo._fileName}{fileInfo._extension}": fileInfo._hash for fileInfo in fileInfoList}

        # 해시값을 통한 파일 도메인 객체 참조
        prevFiles = set(snapHashDict.keys())
        currentFiles = set(currentHashDict.keys())

        # 추가된 파일과 삭제된 파일 검사
        added_files = currentFiles - prevFiles
        removed_files = prevFiles - currentFiles

        # 파일이 추가된 경우
        if added_files:
            print("모니터링에서 추가된 파일 : ")
            for filename in added_files:
                file_hash = currentHashDict[filename]
                hashToFileInfo = {fileInfo._hash: fileInfo for fileInfo in fileInfoList}
                print(hashToFileInfo.get(file_hash).__str__())
            print("")

        # 파일이 삭제된 경우
        if removed_files:
            print("모니터링에서 삭제된 파일 : ")
            for filename in removed_files:
                print(filename)

        common_files = prevFiles & currentFiles
        changed_files = [f for f in common_files if snapHashDict[f] != currentHashDict[f]]

        # 파일이 변경된 경우
        if changed_files:
            print("모니터링중 변경된 파일 : ")
            for filename in changed_files:
                file_hash = currentHashDict[filename]
                hashToFileInfo = {fileInfo._hash: fileInfo for fileInfo in fileInfoList}
                print(hashToFileInfo.get(file_hash).__str__())

        # 딕셔너리 최신화
        snapHashDict = currentHashDict

        # 루프 타이머
        time.sleep(env.interval)
###############################################

# 애플리케이션 실행
main()


디렉터리 감시를 통한 보안 위협 탐지 프로그램 - 장상훈
모니터링에서 추가된 파일 : 
test_5.py | 상태: 주의파일(Python 파일) / 민감정보탐지(주석 포함) 
test_6.js | 상태: 주의파일(JavaScript 파일) / 민감정보탐지(주석 포함, 이메일 포함) 
test_4.txt | 상태: 민감정보탐지(SQL 문장 포함) 
test_7.class | 상태: 주의파일(Class 파일) / 민감정보탐지(주석 포함, 이메일 포함, SQL 문장 포함) 
test_2.txt | 상태: 민감정보탐지(주석 포함) 
test_3.txt | 상태: 민감정보탐지(이메일 포함) 
test_1.txt | 상태: 정상 

모니터링에서 추가된 파일 : 
testtesttest.txt | 상태: 정상 

모니터링중 변경된 파일 : 
testtesttest.txt | 상태: 민감정보탐지(주석 포함) 
모니터링중 변경된 파일 : 
testtesttest.txt | 상태: 민감정보탐지(주석 포함, 이메일 포함, SQL 문장 포함) 
모니터링에서 삭제된 파일 : 
testtesttest.txt


KeyboardInterrupt: 